In [3]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession,SQLContext

In [4]:
spark = SparkContext('local[2]','assignment-2')
sqlContext = SQLContext(spark)

In [5]:
spark_Data = SparkSession.builder.appName("app1").getOrCreate()
df2 = spark_Data.read.csv("./Desktop/sample/aadhaar_data.csv",inferSchema=True,header=True)

In [ ]:
from pyspark.sql.functions import StructType,StringType,StructField,IntegerType,DateType

schema = StructType([
    StructField("date", DateType()),
    StructField("register", StringType()),
    StructField("private_agency", IntegerType()),
    StructField("state", StringType()),
    StructField("distinct", StringType()),
    StructField("sub_distinct", IntegerType()),
    StructField("pincode", IntegerType()),
    StructField("gender", StringType()),
    StructField("age", IntegerType()),
    StructField("aadhar_generated", IntegerType()),
    StructField("rejected", IntegerType()),
    StructField("mobile_number", IntegerType()),
    StructField("email_id", StringType())
])


In [13]:
aadhar_Data = sqlContext.read.csv("./Desktop/sample/aadhaar_data.csv",inferSchema=True,header=False)

In [14]:
aadhar_Data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)



In [29]:
from pyspark.sql.functions import format_number,format_string
df2 =aadhar_Data.select(aadhar_Data['_c0'].alias("date"),
          aadhar_Data['_c1'].alias("register"),
          aadhar_Data['_c2'].alias("private_agency"),
          aadhar_Data['_c3'].alias("state"),
          aadhar_Data['_c4'].alias("distinct"),
          aadhar_Data['_c5'].alias("sub_distinct"),
          aadhar_Data['_c6'].alias("pincode"),
          aadhar_Data['_c7'].alias("gender"),
          aadhar_Data['_c8'].alias("age"),
          aadhar_Data['_c9'].alias("aadhar_generated"),
          aadhar_Data['_c10'].alias("rejected"),
          aadhar_Data['_c11'].alias("mobile_number"),
          aadhar_Data['_c12'].alias("email_id"),
         )
#df2.select(df2['gender']).show()

In [30]:
df2.registerTempTable("aadhar_Data")

### Find the number of states, districts in each state and sub-districts in each district.


In [31]:
sqlContext.sql("select state,distinct,sub_distinct from aadhar_Data group by state,distinct,sub_distinct limit 10").collect()



[Row(state='Uttar Pradesh', distinct='Rae Bareli', sub_distinct='Raebareli'),
 Row(state='Bihar', distinct='Begusarai', sub_distinct='Balia'),
 Row(state='Bihar', distinct='West Champaran', sub_distinct='Bettiah'),
 Row(state='Gujarat', distinct='Rajkot', sub_distinct='Upleta'),
 Row(state='Uttar Pradesh', distinct='Faizabad', sub_distinct='Rudauli'),
 Row(state='Jharkhand', distinct='Dumka', sub_distinct='Kathikund'),
 Row(state='Punjab', distinct='SAS Nagar (Mohali)', sub_distinct='Kharar'),
 Row(state='West Bengal', distinct='Jalpaiguri', sub_distinct='Jalpaiguri'),
 Row(state='West Bengal', distinct='North 24 Parganas', sub_distinct='Bagda'),
 Row(state='Maharashtra', distinct='Buldhana', sub_distinct='Chikhli')]

###  Find the number of males and females in each state from the table. 

In [39]:

sqlContext.sql("select private_agency,state from aadhar_Data group by state,private_agency limit 10").collect()

[Row(private_agency='Conatus Infocom Pvt. Ltd', state='Haryana'),
 Row(private_agency='FINANCIAL INFORMATION NETWORK', state='West Bengal'),
 Row(private_agency='Vakrangee Softwares Limited', state='Haryana'),
 Row(private_agency='Smart Chip Limited', state='Madhya Pradesh'),
 Row(private_agency='Wipro Ltd', state='Nagaland'),
 Row(private_agency='CSS TECHNERGY LIMITED', state='Jammu and Kashmir'),
 Row(private_agency='Multiwave Innovation', state='Gujarat'),
 Row(private_agency='SREI INFRASTRUCTURE FINANCES L', state='Punjab'),
 Row(private_agency='APOnline Limited', state='Andhra Pradesh'),
 Row(private_agency='RELIGARE SECURITIES LTD', state='Jharkhand')]

### Find top 3 states generating most number of Aadhaar cards?


In [43]:
sqlContext.sql("select count(aadhar_generated) as aadhar_count,state from aadhar_Data group by state order by aadhar_count desc limit 3").collect()




[Row(aadhar_count=285326, state='Maharashtra'),
 Row(aadhar_count=209410, state='Uttar Pradesh'),
 Row(aadhar_count=155556, state='Andhra Pradesh')]

### Find top 3 private agencies generating the most number of Aadhar cards?


In [42]:
sqlContext.sql("select count(aadhar_generated) as aadhar_count,private_agency from aadhar_Data group by private_agency order by aadhar_count desc limit 3").collect()



[Row(aadhar_count=130611, private_agency='Wipro Ltd'),
 Row(aadhar_count=116527, private_agency='Vakrangee Softwares Limited'),
 Row(aadhar_count=89562, private_agency='Karvy Data Management Services')]

### Find the number of residents providing email, mobile number? (Hint: consider non-zero values.)

In [46]:
sqlContext.sql("select count(*) as residents_count from aadhar_Data where mobile_number is not null and email_id is not null ").collect()



[Row(residents_count=1665947)]

### Find top 3 districts where enrolment numbers are maximum?

In [48]:
sqlContext.sql("select distinct,count(*) as enrollment_count from aadhar_Data where mobile_number is not null and email_id is not null group by distinct order by enrollment_count desc limit 3 ").collect()



[Row(distinct='Pune', enrollment_count=36203),
 Row(distinct='Mumbai', enrollment_count=29268),
 Row(distinct='Thane', enrollment_count=27848)]

### Find the no. of Aadhaar cards generated in each state?

In [50]:
sqlContext.sql("select state,count(aadhar_generated) as aadhar_generated_count from aadhar_Data group by state order by aadhar_generated_count desc  ").collect()


[Row(state='Maharashtra', aadhar_generated_count=285326),
 Row(state='Uttar Pradesh', aadhar_generated_count=209410),
 Row(state='Andhra Pradesh', aadhar_generated_count=155556),
 Row(state='Rajasthan', aadhar_generated_count=136194),
 Row(state='Bihar', aadhar_generated_count=113401),
 Row(state='Madhya Pradesh', aadhar_generated_count=110245),
 Row(state='Karnataka', aadhar_generated_count=89518),
 Row(state='Gujarat', aadhar_generated_count=87165),
 Row(state='Kerala', aadhar_generated_count=86217),
 Row(state='Jharkhand', aadhar_generated_count=63414),
 Row(state='Punjab', aadhar_generated_count=48855),
 Row(state='Haryana', aadhar_generated_count=46926),
 Row(state='Tamil Nadu', aadhar_generated_count=44968),
 Row(state='West Bengal', aadhar_generated_count=42755),
 Row(state='Delhi', aadhar_generated_count=36917),
 Row(state='Chhattisgarh', aadhar_generated_count=23540),
 Row(state='Odisha', aadhar_generated_count=23216),
 Row(state='Himachal Pradesh', aadhar_generated_count=1913

### Find the number of unique pincodes in the data? 

In [52]:
sqlContext.sql("select distinct count(pincode) from aadhar_Data").collect()


[Row(count(pincode)=1665947)]

### Find the number of Aadhaar registrations rejected in Uttar Pradesh and Maharashtra?

In [54]:
sqlContext.sql("select count(rejected) as rejected_count from aadhar_Data where state in ('Maharashtra','Uttar Pradesh')").collect()

[Row(rejected_count=494736)]

### The top 3 states where the percentage of Aadhaar cards being generated for males is the highest.

In [55]:
sqlContext.sql("select state,count(aadhar_generated) as male_Aadhar_count from aadhar_Data where gender='M'group by state order by male_Aadhar_count desc limit 3  ").collect()




[Row(state='Maharashtra', male_Aadhar_count=147255),
 Row(state='Uttar Pradesh', male_Aadhar_count=104209),
 Row(state='Andhra Pradesh', male_Aadhar_count=74748)]

### In each of these 3 states, identify the top 3 districts where the percentage of Aadhaar cards being rejected for fe 

In [58]:
sqlContext.sql("select state,distinct,count(rejected) over(partition by state,distinct order by state,distinct desc ) female_aadhar_reject from aadhar_Data where gender='F' order by female_aadhar_reject desc limit 30  ").collect()





[Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=16919),
 Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=16919),
 Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=16919),
 Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=16919),
 Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=16919),
 Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=16919),
 Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=16919),
 Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=16919),
 Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=16919),
 Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=16919),
 Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=16919),
 Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=16919),
 Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=16919),
 Row(state='Maharashtra', distinct='Pune', female_aadhar_reject=

### Write a command to see the correlation between “age” and “mobile_number”? (Hint: Consider the percentage of people who have provided the mobile number out of the total applicants) 

In [59]:
from pyspark.sql.functions import corr
df2.select(corr('age','mobile_number')).show()

+------------------------+
|corr(age, mobile_number)|
+------------------------+
|    -0.03104628010634...|
+------------------------+

